In [1]:
!sudo apt-get update
!sudo apt-get install tesseract-ocr
!pip install pytesseract
!pip install easyocr
import easyocr
reader = easyocr.Reader(['en'])

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [57.5 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,310 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,150 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,575 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,030 kB]
Get:14 http://s

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

In [2]:
import pandas as pd
train = pd.read_csv('train.csv')
train.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [27]:
from PIL import Image
import pytesseract
import re
import numpy as np
import easyocr
import requests
from PIL import Image
import pytesseract
from io import BytesIO
import re

entity_unit_map = {
    'width': {'centimetre', 'cm', 'foot', 'ft', 'inch', 'in', 'metre', 'm', 'millimetre', 'mm', 'yard', 'yd'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'g', 'kilogram', 'kg', 'microgram', 'µg', 'milligram', 'mg', 'ounce','oz', 'pound', 'lb', 'ton', 't'},
    'maximum_weight_recommendation': {'gram', 'g', 'kilogram', 'kg', 'microgram', 'µg', 'milligram', 'mg', 'ounce','oz', 'pound', 'lb', 'ton', 't'},
    'voltage': {'kilovolt', 'kV', 'millivolt', 'mV', 'volt' 'V'},
    'wattage': {'kilowatt', 'kW', 'watt' 'W'},
    'item_volume': {'centilitre', 'cl', 'cubic foot', 'ft³', 'cubic inch', 'in³', 'cup', 'decilitre', 'dl', 'fluid ounce', 'fl oz', 'gallon', 'gal',
                    'imperial gallon', 'imp gal', 'litre', 'L', 'microlitre', 'µL', 'millilitre', 'mL', 'pint', 'pt', 'quart', 'qt'}
}

unit_abbreviations = {
    'cm': 'centimetre',
    'm': 'metre',
    'mm': 'millimetre',
    'ft': 'foot',
    'in': 'inch',
    'yd': 'yard',
    'g': 'gram',
    'kg': 'kilogram',
    'µg': 'microgram',
    'mg': 'milligram',
    'oz': 'ounce',
    'lb': 'pound',
    't': 'ton',
    'kV': 'kilovolt',
    'mV': 'millivolt',
    'V': 'volt',
    'kW': 'kilowatt',
    'W': 'watt',
    'cl': 'centilitre',
    'ft³': 'cubic foot',
    'in³': 'cubic inch',
    'cup': 'cup',
    'dl': 'decilitre',
    'fl oz': 'fluid ounce',
    'gal': 'gallon',
    'imp gal': 'imperial gallon',
    'l': 'litre',
    'µl': 'microlitre',
    'ml': 'millilitre',
    'pt': 'pint',
    'qt': 'quart'
}

def convert_unit(match):
    pattern = re.compile(r"([0-9.]+)\s*([a-zA-Zµ]+)")

    def replacer(m):
        number = m.group(1)
        unit = m.group(2).lower()
        full_word = unit_abbreviations.get(unit, unit)

        return f"{number} {full_word}"

    return pattern.sub(replacer, match)


def convert_all_matches(all_matches):
    return [convert_unit(match) for match in all_matches]

def check_for_units(key, text_lines, entity_unit_map):
    units = entity_unit_map.get(key, set())
    pattern = r'(\d+\.?\d*)\s*-\s*(\d+\.?\d*)?\s*(' + '|'.join(re.escape(unit) for unit in units) + r')|' + \
              r'(\d+\.?\d*)\s*(' + '|'.join(re.escape(unit) for unit in units) + r')'

    matches = []
    for line in text_lines:
        for match in re.finditer(pattern, line, re.IGNORECASE):
            value = match.group(1) or match.group(4)
            if value and float(value) != 0:
                matches.append(match.group(0))
    return matches

def extract_text_from_image(image):
    return pytesseract.image_to_string(image)



def download_image_from_url(image_url):
    try:
        response = requests.get(image_url)
        response.raise_for_status()
        img = Image.open(BytesIO(response.content))
        return img
    except Exception as e:
        print(f"Error processing {image_url}: {e}")
        return None

def process_image(image_url, entity_name):
    img = download_image_from_url(image_url)
    if img is None:
        return None
    angles = [0, 180, 45, 315, 90, 270, 135, 225]
    all_matches = []
    for angle in angles:
        rotated_img = img.rotate(angle, expand=True)
        text = extract_text_from_image(rotated_img)
        lines = text.split('\n')
        extracted_texts = check_for_units(entity_name, lines, entity_unit_map)
        if extracted_texts:
            all_matches.extend(extracted_texts)
    if not all_matches:
        for angle in angles:
            rotated_img = img.rotate(angle, expand=True)
            rotated_img_np = np.array(rotated_img)
            results = reader.readtext(rotated_img_np)
            extracted_texts = check_for_units(entity_name, [result[1] for result in results], entity_unit_map)
            if extracted_texts:
                all_matches.extend(extracted_texts)
                break

    return sorted(set(all_matches))

def separate_based_on_unit(matches):
    unit_dict = {}
    pattern = re.compile(r"([0-9.]+)\s*([a-zA-Zµ ]+)")
    for match in matches:
        result = pattern.match(match)
        if result:
            number = float(result.group(1))
            unit = result.group(2).strip()

            if unit not in unit_dict:
                unit_dict[unit] = []
            unit_dict[unit].append(number)

    return unit_dict

def process_and_compare(row):
    image_url = row['image_link']
    entity_name = row['entity_name']
    entity_value = row['entity_value']

    try:
        all_matches = process_image(image_url, entity_name)
        all_matches = convert_all_matches(all_matches)
        separated_units = separate_based_on_unit(all_matches)
        print(f"Entity Name: {entity_name}")
        print(f"Entity Value: {entity_value}")
        print(f"All Matches:")
        for unit, values in separated_units.items():
            print(f"{unit}: {values}")
        print()



    except Exception as e:
        print(f"Error processing {image_url}: {e}\n")

for index, row in train.head(5).iterrows():
    print("Index:",index)
    process_and_compare(row)



Index: 0
Entity Name: item_weight
Entity Value: 500.0 gram
All Matches:
gram: [500.0]

Index: 1
Entity Name: item_volume
Entity Value: 1.0 cup
All Matches:
litre: [3.0]

Index: 2
Entity Name: item_weight
Entity Value: 0.709 gram
All Matches:
gram: [0.09, 0.709, 2.0, 51.0, 1.0, 7.0]
ton: [1.0]
milligram: [10.0, 100.0, 200.0, 25.0, 50.0]

Index: 3
Entity Name: item_weight
Entity Value: 0.709 gram
All Matches:
gram: [0.709, 553.0]
ton: [1.0, 102.0]

Index: 4
Entity Name: item_weight
Entity Value: 1400 milligram
All Matches:
milligram: [1400.0]

